In [13]:
import math, copy
import multiprocessing as mp

In [14]:
size = range(2, 50)

In [3]:
# Part 1: Imagine a circle of elves, numbered from 1, each holding a gift.
# Going around the circle in order, each elf steals a gift from the elf on
# their left. Any elf without a gift is removed from the circle. Which elf
# ends up with all the gifts?

# Solution: First I solved this problem by walking the list, which was
# prohibitively slow for large n
def part1a(x):
    p1 = {}
    elves = list(range(x))
    while len(elves) > 1:
        for i, e in enumerate(elves):
            elves.pop((i+1) % len(elves))
    p1[x] = elves[0] + 1
    return p1

In [4]:
# I tried speeding up execution with threading or multiprocessing, but
# concluded that the computation is CPU-bound and modifies a single massive
# data structure - I would have to chunk it manually. Along the way, I ran
# the above code for n from 2 to 50. Noticing a pattern in the output, I
# derived a formula for both results.
    
# part1: 2^n => 1, then follow 2^n odd numbers (so the last is 2^n+1 - 1 => 2^n+1 - 1)

def part1b(x):
    powers = [2**n for n in range(25)]
    odds = [1 + 2*n for n in range(x)]
    
    t = [a for a in powers if a <= x]
    r = x - t[-1]
    return {x: odds[r]}

In [5]:
with mp.Pool() as pool:
    a = pool.map(part1a, size)
    b = pool.map(part1b, size)
a == b

True

In [6]:
%%time
part1b(3001330)

CPU times: user 475 ms, sys: 70.6 ms, total: 546 ms
Wall time: 577 ms


{3001330: 1808357}

In [15]:
# Part 2: Imagine instead that the elves steal gifts from the elf directly
# across the circle from them, or the elf on the left (from the stealer's
# view) if there are two options. Now which elf ends up with all the gifts?

# Solution: Again, this code was technically correct but incredibly slow; it would've run for 24+ hrs on the problem input.

def part2a(x):
    p2 = {}
    elves = list(range(x))
    while len(elves) > 1:
        ref = copy.deepcopy(elves)
        for r in ref:
            if r in elves:
                j = elves.index(r)
                elves.pop((j + math.floor(len(elves) / 2)) % len(elves))
    p2[x] = elves[0] + 1
    
    return p2

In [16]:
# part2: 1 => 1, if n => n then start 1, 2, 3 until n => n/2, then continue sequence of odds until n => n

# This still takes a while to run
def part2b(x):
    p2 = {}
    odds = [1 + 2*n for n in range(x)]
    numbers = odds
    for n in range(1, x + 1):
        o = numbers[0]
        if n == x:
            p2[n] = o

        numbers.pop(0)
        if n == o:
            numbers = list(range(1, n + 1))
        if n/2 == o:
            numbers = odds

    return p2

In [17]:
# By inspection, the values of n where n -> n are powers of 3.
def part2c(x):
    odds = [1 + 2*n for n in range(x)]
    
    for r in range(15):
        if 3**r >= x:
            a = r - 1
            break
    
    # Find the power of 3 below x, its difference with x, and its difference with the next power of 3
    p = 3**a
    r = x - p
    d = 3**(a+1) - p
    
    # if the remainder is less than half the distance to the next power of 3, y is r
    if r <= d//2:
        y = r
    # otherwise pick y out of the sequence of odds
    else:
        s = r - d//2
        i = odds.index(p)
        y = odds[i + s]

    return {x: y}

In [18]:
with mp.Pool() as pool:
    a = pool.map(part2a, size)
    b = pool.map(part2b, size)
    c = pool.map(part2c, size)
    print(c)

a == b and b == c

[{2: 1}, {3: 3}, {4: 1}, {5: 2}, {6: 3}, {7: 5}, {8: 7}, {9: 9}, {10: 1}, {11: 2}, {12: 3}, {13: 4}, {14: 5}, {15: 6}, {16: 7}, {17: 8}, {18: 9}, {19: 11}, {20: 13}, {21: 15}, {22: 17}, {23: 19}, {24: 21}, {25: 23}, {26: 25}, {27: 27}, {28: 1}, {29: 2}, {30: 3}, {31: 4}, {32: 5}, {33: 6}, {34: 7}, {35: 8}, {36: 9}, {37: 10}, {38: 11}, {39: 12}, {40: 13}, {41: 14}, {42: 15}, {43: 16}, {44: 17}, {45: 18}, {46: 19}, {47: 20}, {48: 21}, {49: 22}]


True

In [11]:
%%time
# part2a(300) -> 3ms
# part2a(3000) -> 166ms
# part2a(30000) -> 15s
# part2a(300000) -> 18min

CPU times: user 16min 13s, sys: 4.5 s, total: 16min 17s
Wall time: 17min 42s


{300000: 122853}

In [12]:
%%time
part2b(3001330)

CPU times: user 42min 24s, sys: 12.2 s, total: 42min 36s
Wall time: 1h 1min


{3001330: 1407007}

In [43]:
%%time
part2c(3001330)

CPU times: user 521 ms, sys: 77.8 ms, total: 599 ms
Wall time: 1.09 s


{3001330: 1407007}